In [ ]:
import cmipld
from cmipld.utils.offline import LD_server, replace_loader
from pyld import jsonld

In [ ]:
from pyld import jsonld 
jsonld.expand('https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product.json')

In [1]:
import cmipld
cmipld.processor.expand_document('https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product.json')

/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


xccc https://wcrp-cmip.github.io/WCRP-universe/product/_context_ {'https://wcrp-cmip.github.io/WCRP-universe/product/description': [{'@value': 'Output produced from a model simulation. This includes the various data points and metrics generated as a result of running climate models, which simulate physical, chemical, and biological processes affecting the climate system.'}], '@id': 'https://wcrp-cmip.github.io/WCRP-universe/product/model-output', 'https://wcrp-cmip.github.io/WCRP-universe/product/label': [{'@value': 'model-output'}], '@type': ['wcrp:product', 'https://wcrp-cmip.github.io/WCRP-universe/']}
[{'@context': 'https://wcrp-cmip.github.io/WCRP-universe/product/_context_', 'id': 'universal:product/model-output', 'type': ['wcrp:product', 'universal'], 'description': 'Output produced from a model simulation. This includes the various data points and metrics generated as a result of running climate models, which simulate physical, chemical, and biological processes affecting the c

[{'@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/_context_',
  'id': 'cmip6plus:project/product',
  'type': ['wcrp:project', 'cmip6plus', 'wcrp:product'],
  'product': {'id': 'universal:product/model-output',
   'type': ['wcrp:product', 'universal'],
   'universal:product/description': 'Output produced from a model simulation. This includes the various data points and metrics generated as a result of running climate models, which simulate physical, chemical, and biological processes affecting the climate system.',
   'universal:product/label': 'model-output'}}]

In [ ]:
from pyld import jsonld
jsonld.expand(f'https://wcrp-cmip.github.io/WCRP-universe/product/model-output.json')

In [ ]:
jsonld.expand('https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product.json')

In [ ]:
# Set up offline override for the current repository

ldpath = cmipld.utils.git.ldpath()


# create a temporary directory containing (a copy of) the current repository
# localserver = LD_server(copy = [[ldpath,'current_repo']])
repos= { 
        'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
        'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
        'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
        }

localserver = LD_server(repos = repos.items(),copy = [[ldpath,'cmip6plus']])

# localserver = LD_server(copy = [[ldpath,'cmip6plus']])


localhost = localserver.start_server(8082)
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)





In [ ]:
# 
replace_loader(localhost,[list(i) for i in repos.items()])


In [ ]:
cmipld.processor.expand_document(f'{io_url}project/{item}.json',depth=4)

In [ ]:
f'{io_url}project/{item}.json'

In [ ]:
jsonld.expand(f'{io_url}project/{item}.json')

In [ ]:
jsonld.expand(f'https://wcrp-cmip.github.io/WCRP-universe/product/model-output.json')

In [ ]:
jsonld.frame(f'https://wcrp-cmip.github.io/WCRP-universe/product/model-output.json',f'https://wcrp-cmip.github.io/WCRP-universe/product/model-output.json')

In [ ]:
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

# localserver.copy_existing_repos([[ldpath,'cmip6plus']])

item='product'
a = jsonld.expand(f'{io_url}project/{item}.json')



b = jsonld.frame(f'{io_url}project/{item}.json',f'{io_url}project/_context_')

a

In [3]:


def get_entry(data,entry='label'):
    return [i.get(entry) for i in data]

def tables(data):
    return get_entry(data['tables'])
    
# getattr(cmipld,'processor')

In [5]:
# getattr(cmipld,'processor')

In [ ]:
from collections import OrderedDict

CV = OrderedDict()


for item in [
    'product',
    'mip-era',
    'license',
    'activity',
    'tracking-id',
    'drs',
    'repo',
    # 'tables',
    'required-global-attributes',
    'index'
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    CV[item] = cmipld.processor.get(f'{io_url}project/{item}.json', depth=3)[0]



In [ ]:
CV['product']

In [ ]:


# cmipld.processor.loader(f'{io_url}project/_context_',{})

jsonld.expand([{
        "@context": [
        {
            "@base": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/",
            "@vocab": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/",
            "activity": {
                "@container": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/activity/_context_"
            },
            "mip_era": {
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/mip/_context_"
            },
            "product": {
                "@type": "@id",
                "@context": "https://wcrp-cmip.github.io/WCRP-universe/product/_context_"
            },
            "tables": {
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/MIP-variables/tables/_context_"
            },
            "license":{
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/license/_context_"
            }
        }
    ],
        '@id': 'project/product',
  'product': ['model-output'],
  '@type': ['wcrp:project',
   'https://wcrp-cmip.github.io/CMIP6Plus_CVs/',
   'wcrp:product']}])#,f'{io_url}project/product.json')



In [ ]:
import requests
item='product'
print(io_url)
iourl = localhost
d = requests.get(f'{io_url}project/{item}.json').json()
c = requests.get(f'{io_url}project/_context_').json()
c

In [ ]:
d['@context'] = c['@context'][1]
jsonld.expand(d)

In [ ]:

# localserver.copy_existing_repos([[ldpath,'cmip6plus']])

item='product'
jsonld.expand(f'{io_url}project/{item}.json')

In [ ]:
# This means we can make 'locally hosted' content appear as an external resource in our PYLD instance

testfile = f'{io_url}project/'



In [ ]:
CV['license']

In [ ]:
item = 'tables'




frame = {
    '@context': [
   {'@base': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
    '@vocab': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
    'activity': {'@container': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'},
    'mip_era': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/mip/_context_'},
    'product': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/product/_context_'},
    'tables': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/_context_'},
    'license': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/license/_context_'}}],
    "id": "product",
    "type": [
        "wcrp:project",
        "cmip6plus",
        "wcrp:product"
    ],
    "product": [
        "model-output"
    ]
         
}
         
         
        #  ,"tables":[{"product":""}],"@embed":"@always"}

res = jsonld.expand(frame)

res

In [ ]:
#  if depth = 1 parallel, otherwise serial 
#   ptqdm?


cahcheing 
fail repeat 



In [ ]:
from p_tqdm import p_map

In [ ]:
from pyld import jsonld
jsonld.frame(cmipld.get(f'{io_url}project/{item}.json'), {
    '@context': [
        f'{io_url}project/{item}.json', 
        ],
    "tables":{
            "table_id":'',
            "@explicit": True,
            }
    })

In [ ]:
jsonld.frame(f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{"@context":[f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{}],"table_id":{},"@explicit":True})

In [ ]:
# localserver.copy_existing_repos([[ldpath,'current_repo']])

In [ ]:
cmipld.get(f'{io_url}source/cesm2.json')

In [ ]:
           "model-component": {
                "@context": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/component/_context_",
                "@type": "@id"
            }

In [ ]:
jsonld.expand(f'{io_url}source/cesm2.json')

In [3]:
import glob
fls = glob.glob('*.json')
fls

import json
ctx = json.load(open('_context_'))['@context'][1]
ctx

{'@base': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
 '@vocab': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
 'activity': {'@container': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'},
 'mip_era': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/mip/_context_'},
 'tables': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/_context_'},
 'license': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/license/_context_'},
 'product': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/product/_context_'}}

In [4]:
for i,j in ctx.items():
    if '@' not in i:
        data= json.load(open(f"{i.replace('_','-')}.json"))
        data['@context'] = ['_context_',j]
        print(data)
        json.dump(data,open(f"{i.replace('_','-')}.json",'w'),indent=4)

{'@context': ['_context_', {'@container': '@id', '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'}], 'id': 'activity', 'type': ['wcrp:project', 'cmip6plus', 'wcrp:activity'], 'activity': {'lesfmip': {'start': 2024}, 'cmip': {'start': 2023}}}
{'@context': ['_context_', {'@type': '@id', '@context': 'https://wcrp-cmip.github.io/WCRP-universe/mip/_context_'}], 'id': 'mip-era', 'type': ['wcrp:project', 'cmip6plus', 'wcrp:mip_era'], 'mip_era': ['cmip6plus']}
{'@context': ['_context_', {'@type': '@id', '@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/_context_'}], 'id': 'tables', 'type': ['wcrp:project', 'cmip6plus', 'wcrp:tables'], 'tables': ['acmon', 'acmonz', 'ae1hr', 'ae3hrpt', 'ae3hrptlev', 'ae6hr', 'ae6hrpt', 'ae6hrptlev', 'aeday', 'aemon', 'aemonlev', 'aemonz', 'aesubhrpt', 'aesubhrptsite', 'ap1hr', 'ap1hrpt', 'ap3hr', 'ap3hrpt', 'ap3hrptlev', 'ap6hr', 'ap6hrpt', 'ap6hrptlev', 'ap6hrptz', 'apday', 'apdaylev', 'apdayz', 'apfx', 'apmon', 'apmonclim'